In [1]:
import pandas as pd

In [3]:
berkeley = pd.read_csv('berkeley_prefs.csv')
berkeley

,First,Last,School,Online,Rounds,Rating
0,Nihar,Abhyankar,Liberal Arts and Science Academy,NaN,6,2.0
1,Taha,Amir,Hire,NaN,6,5.0
2,Buck,Arney,Sonoma Academy,NaN,3,1.0
3,Tristan,Bato,College Prep,NaN,3,NaN
4,Alex,Brown,Montgomery Bell Academy,NaN,2,2.0
...,...,...,...,...,...,...
87,Coralynn,Yang,Peninsula High School,NaN,6,3.0
88,Kelly,Ye,College Prep,NaN,3,NaN
89,Jack,Young,Montgomery Bell Academy,NaN,5,3.0
90,Iris,Zhang,The Quarry Lane School,NaN,6,3.0


In [11]:
berkeley.drop(['School', 'Online', 'Rounds'], axis=1, inplace=True)
# berkeley.columns
berkeley

,First,Last,Rating
0,Nihar,Abhyankar,2.0
1,Taha,Amir,5.0
2,Buck,Arney,1.0
3,Tristan,Bato,NaN
4,Alex,Brown,2.0
...,...,...,...
87,Coralynn,Yang,3.0
88,Kelly,Ye,NaN
89,Jack,Young,3.0
90,Iris,Zhang,3.0


In [52]:
import mysql.connector
cnx = mysql.connector.connect(user='root', password='', host='localhost', database='pref-buddy', port=3306)
cursor = cnx.cursor()
cursor

In [32]:
cursor.execute("SELECT f_name, l_name, id FROM users")
users = cursor.fetchall()
users_df = pd.DataFrame(users, columns=['f_name', 'l_name', 'id'])
users_df

,f_name,l_name,id
0,Kelly,Ye,0
1,Chris,Palmer,1
2,Ian,Beier,2
3,Debnil,Sur,5
4,Kelly,Ye,9
...,...,...,...
438,Ajay,Umaranikar,736689
439,Jayati,Hazra,775841
440,Arjun,Amin,775842
441,John,Lee,780947


In [33]:
berkeley = pd.merge(left=berkeley, right=users_df, how='left', left_on=['First', 'Last'], right_on=['f_name', 'l_name'])
berkeley

,First,Last,Rating,f_name,l_name,id
0,Nihar,Abhyankar,2.0,Nihar,Abhyankar,235724.0
1,Taha,Amir,5.0,Taha,Amir,218590.0
2,Buck,Arney,1.0,Buck,Arney,178280.0
3,Tristan,Bato,NaN,Tristan,Bato,62341.0
4,Alex,Brown,2.0,Alex,Brown,11718.0
...,...,...,...,...,...,...
90,Kelly,Ye,NaN,Kelly,Ye,9.0
91,Kelly,Ye,NaN,Kelly,Ye,76128.0
92,Jack,Young,3.0,Jack,Young,88145.0
93,Iris,Zhang,3.0,Iris,Zhang,300704.0


In [34]:
berkeley.dropna(axis=0, inplace=True)
berkeley.drop(['First', 'Last', 'f_name', 'l_name'], axis=1, inplace=True)

In [35]:
berkeley

,Rating,id
0,2.0,235724.0
1,5.0,218590.0
2,1.0,178280.0
4,2.0,11718.0
5,2.0,238559.0
...,...,...
87,2.0,148389.0
88,3.0,183389.0
92,3.0,88145.0
93,3.0,300704.0


In [41]:
arjun_id = [775842 for _ in range(len(berkeley))]

berkeley['ranker_id'] = arjun_id
berkeley

,Rating,id,ranker_id
0,2.0,235724.0,775842
1,5.0,218590.0,775842
2,1.0,178280.0,775842
4,2.0,11718.0,775842
5,2.0,238559.0,775842
...,...,...,...
87,2.0,148389.0,775842
88,3.0,183389.0,775842
92,3.0,88145.0,775842
93,3.0,300704.0,775842


In [43]:
berkeley.rename({'Rating': 'rating', 'id': 'judge_id'}, axis=1, inplace=True)
berkeley

,rating,judge_id,ranker_id
0,2.0,235724.0,775842
1,5.0,218590.0,775842
2,1.0,178280.0,775842
4,2.0,11718.0,775842
5,2.0,238559.0,775842
...,...,...,...
87,2.0,148389.0,775842
88,3.0,183389.0,775842
92,3.0,88145.0,775842
93,3.0,300704.0,775842


In [57]:
insert_sql = ("""
    INSERT into ranks (rating, judge_id, ranker_id)
    VALUES (%(rating)s, %(judge_id)s, %(ranker_id)s)
    ON DUPLICATE KEY UPDATE
    rating = VALUES(rating)
""")

In [58]:
for i in range(len(berkeley)):
    # print(berkeley.iloc[i].to_json())
    try:
        row = berkeley.iloc[i].to_dict()
        cursor.execute(insert_sql, row)
        cnx.commit()
    except mysql.connector.Error as err:
        cnx.rollback()
        print(err)
